In [1]:
!pip install wikipedia
!pip install surprise
!pip install ipyplot

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=888592202a38134b975402f9f6c8125dd0cf7deec89cd142b25bf720a7867f6d
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163754 sha256=d192dd4d30fd916f0f577e5c5d37e34a162feb2161a768b94ae783020e3aa363
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/MyDrive/Colab Notebooks'

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!unzip 'archive.zip'
!unzip 'ml-1m.zip'
!unzip 'archive (1).zip'

unzip:  cannot find or open archive.zip, archive.zip.zip or archive.zip.ZIP.
unzip:  cannot find or open ml-1m.zip, ml-1m.zip.zip or ml-1m.zip.ZIP.
unzip:  cannot find or open archive (1).zip, archive (1).zip.zip or archive (1).zip.ZIP.


In [ ]:
!ls

CAERS


In [ ]:
import pandas as pd
import plotly.express as px
from sklearn.feature_extraction.text import TfidfVectorizer
import wikipedia
import numpy as np
from tqdm.notebook import tqdm
import re
import os
from sklearn.metrics.pairwise import linear_kernel
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_selection import VarianceThreshold
import requests
from io import BytesIO
import nltk
import ipyplot

tqdm.pandas()


        WARNING! Google Colab Environment detected!
        You might encounter issues while running in Google Colab environment.
        If images are not displaying properly please try setting `force_b64` param to `True`.
        


#Movie DataSet

In [ ]:
movies_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CAERS/ml-1m/movies.dat',
                        encoding = "ISO-8859-1",
                        delimiter='::', engine= 'python', header=None,
                        names=['movie_name', 'genre'])




In [ ]:
movies_df.head()

,movie_name,genre
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride Part II (1995),Comedy


In [ ]:
movies_df['genre'] = movies_df.genre.str.split('|')

In [ ]:
p= wikipedia.page('Toy Story (1998)')
s = p.content

In [ ]:
def get_wikipedia_page_name(raw_name):
    names = wikipedia.search(raw_name)
    if len(names) == 0:
      return ''
    else:
      return names[0]


In [ ]:
def get_movie_plot(page_name):
    try:
      try:
        movie_page_content = str(wikipedia.page(page_name, auto_suggest=False).content)
      except wikipedia.DisambiguationError as e:
        for option in e.options:
          if 'film' in option:
            movie_page_content = str(wikipedia.page(option, auto_suggest=False).content)
        return np.nan
    except (wikipedia.PageError, KeyError):
      return np.nan
    re_groups = re.search("Plot ==(.*?)=+ [A-Z]", str(movie_page_content).replace('\n', ''))
    if re_groups:
      return re_groups.group(1)
    else:
      return np.nan

In [ ]:

movies_df['wikipedia_page_name'] = movies_df['movie_name'].progress_apply(lambda name: get_wikipedia_page_name(name))
movies_df['movie_plot'] = movies_df['wikipedia_page_name'].progress_apply(lambda page_name: get_movie_plot(page_name))
print(f'There are {movies_df["movie_plot"].isna().sum()} NaN movie plots')
movies_df[['movie_name', 'movie_plot']].head()


  0%|          | 0/3883 [00:00<?, ?it/s]

  0%|          | 0/3883 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


There are 685 NaN movie plots


,movie_name,movie_plot
1,Toy Story (1995),A group of living toys are preparing to move i...
2,Jumanji (1995),"In 1969, Alan Parrish lives with his parents, ..."
3,Grumpier Old Men (1995),The feud between Max and John has cooled and t...
4,Waiting to Exhale (1995),"Four friends (Savannah, Robin, Bernadine, and ..."
5,Father of the Bride Part II (1995),The film begins four years after the events of...


In [ ]:
movies_df.head()

,movie_name,genre,wikipedia_page_name,movie_plot
1,Toy Story (1995),"[Animation, Children's, Comedy]",Toy Story,A group of living toys are preparing to move i...
2,Jumanji (1995),"[Adventure, Children's, Fantasy]",Jumanji,"In 1969, Alan Parrish lives with his parents, ..."
3,Grumpier Old Men (1995),"[Comedy, Romance]",Grumpier Old Men,The feud between Max and John has cooled and t...
4,Waiting to Exhale (1995),"[Comedy, Drama]",Waiting to Exhale,"Four friends (Savannah, Robin, Bernadine, and ..."
5,Father of the Bride Part II (1995),[Comedy],Father of the Bride Part II,The film begins four years after the events of...


In [ ]:
movies_df.to_pickle('/content/drive/MyDrive/Colab Notebooks/movie.csv')

#User Dataset


In [ ]:
readme_text = np.array(open('/content/drive/MyDrive/Colab Notebooks/CAERS/ml-1m/README').read().splitlines())
start_index = np.flatnonzero(np.core.defchararray.find(readme_text,'Occupation is chosen')!=-1)[0]
end_index = np.flatnonzero(np.core.defchararray.find(readme_text,'MOVIES FILE DESCRIPTION')!=-1)[0]
occupation_list = [x.split('"')[1] for x in readme_text[start_index:end_index][2:-1].tolist()]
occupation_dict = dict(zip(range(len(occupation_list)), occupation_list))

users_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CAERS/ml-1m/users.dat',
                       delimiter='::', engine='python', header=None,
                       names=['user_id', 'gender', 'age', 'occupation', 'zip_code'])
users_df['occupation'] = users_df['occupation'].replace(occupation_dict)
users_df.head()

,user_id,gender,age,occupation,zip_code
0,1,F,1,K-12 student,48067
1,2,M,56,self-employed,70072
2,3,M,25,scientist,55117
3,4,M,45,executive/managerial,02460
4,5,M,25,writer,55455


In [ ]:
users_df.to_pickle('/content/drive/MyDrive/Colab Notebooks/user.csv')

#Rating

In [ ]:
ratings_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CAERS/ml-1m/ratings.dat',
                         delimiter='::', engine='python', header=None,
                         names=['user_id', 'movie_id', 'rating', 'time'])
ratings_df.head()

,user_id,movie_id,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
ratings_df.to_pickle('/content/drive/MyDrive/Colab Notebooks/rating.csv')